In [5]:
import math
import pandas as pd
import numpy as np
from pprint import pprint

from scipy import constants
from astropy import constants as astro_const
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import pbhs as pbhs
import black_holes as bh

pd.set_option('display.width', 500)

# solar_mass_grams = 1000 * astro_const.M_sun.value

# PBHs

## Quantum evaporation and absorption of energy

Black holes emit thermal radiation at a temperature inversely proportional to their mass. Numerically, this effect is given by

\begin{equation}
T_{bh}=\frac{\hbar{c}^{3}}{8\pi{k_{B}G}M}\simeq \frac{6.2\times{10}^{-8}K}{(M/M_{\odot})}
\end{equation}




In [6]:
solar_mass_units = np.arange(1, 5)

masses_kg = solar_mass_units * astro_const.M_sun.value

# Calculate the Hawking temperature for each mass
hawking_temperatures = [pbhs.hawking_temp_from_solar_mass(m, astro_const.M_sun.value) for m in masses_kg]

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_solar) from import': solar_mass_units,
    'Mass (kg)': masses_kg,
    'Hawking Temperature (K)': hawking_temperatures
})

# Display the table
print(df)

   Mass (M_solar) from import     Mass (kg)  Hawking Temperature (K)
0                           1  1.988410e+30             6.200000e-08
1                           2  3.976820e+30             3.100000e-08
2                           3  5.965230e+30             2.066667e-08
3                           4  7.953639e+30             1.550000e-08


In [7]:
temperature_critical_mass = pbhs.critical_mass_for_bg_temperature(2.7) / pbhs.solar_mass_grams
dmdt, first, second, _ = pbhs.dm_dt(5 * pbhs.solar_mass_grams, 2.7)

print(f"mass_c({temperature_critical_mass} K): {temperature_critical_mass:.2e}")
print(f"dmdt: {dmdt:.2e}")
print(f" - first: {first:.2e}")
print(f" - second: {second:.2e}")


mass_c(2.2910545872296856e-08 K): 2.29e-08
dmdt: 9.19e-11
 - first: -4.01e-44
 - second: 9.19e-11


In [14]:
background_temperature = 2.7
# solar_mass_units = np.arange(1, 100)
solar_mass_units = np.logspace(start=-20, stop=1, num=50, base=10)


temperature_critical_mass = pbhs.critical_mass_for_bg_temperature(background_temperature)
print(96*"-")
print(f"Planck mass (g): {pbhs.planck_mass_g:.2e} grams")
print(f"critical_mass({background_temperature} K): {temperature_critical_mass/pbhs.solar_mass_grams:.2e} \M_sun ({temperature_critical_mass:.2e} grams)")
print(96*"-")

masses_g = solar_mass_units * pbhs.solar_mass_grams
# print("\nMasses(g)")
# pprint(masses_g)

hawking_temperatures = [pbhs.hawking_temp_from_solar_mass(m, pbhs.solar_mass_grams) for m in masses_g]
# print("\nHawking Temperatures(K)")
# pprint(hawking_temperatures)

terms_tuple = [pbhs.dm_dt(m, background_temperature) for m in masses_g]
# print("\nterms_tuple")
# pprint(terms_tuple)

dm_dt_column, first_term_col, second_term_col, error_diff = zip(*terms_tuple)
# print("\ndm_dt_column")
# pprint(dm_dt_column[0])

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_sun)': solar_mass_units,
    'Mass (g)': masses_g,
    'Hawking temp. (K)': hawking_temperatures,
    # 'dM/dt': dm_dt_column,
    'First term (Hawking Evaporation)': first_term_col,
    'Second term (absorption)': second_term_col,
    'Close?': error_diff
})

# Display the table
print(df.to_markdown(floatfmt='.2e', index=False))

------------------------------------------------------------------------------------------------
Planck mass (g): 2.18e-05 grams
critical_mass(2.7 K): 2.29e-08 \M_sun (4.56e+25 grams)
------------------------------------------------------------------------------------------------
|   Mass (M_sun) |   Mass (g) |   Hawking temp. (K) |   First term (Hawking Evaporation) |   Second term (absorption) | Close?   |
|---------------:|-----------:|--------------------:|-----------------------------------:|---------------------------:|:---------|
|       1.00e-20 |   1.99e+13 |            6.20e+12 |                          -1.00e-02 |                   3.68e-52 | False    |
|       2.68e-20 |   5.33e+13 |            2.31e+12 |                          -1.39e-03 |                   2.65e-51 | False    |
|       7.20e-20 |   1.43e+14 |            8.61e+11 |                          -1.93e-04 |                   1.90e-50 | False    |
|       1.93e-19 |   3.84e+14 |            3.21e+11 |           

<>:9: SyntaxWarning: invalid escape sequence '\M'
<>:9: SyntaxWarning: invalid escape sequence '\M'
/var/folders/n5/91crw72j2wl9689p75btgrf40000gn/T/ipykernel_47398/2896665515.py:9: SyntaxWarning: invalid escape sequence '\M'
  print(f"critical_mass({background_temperature} K): {temperature_critical_mass/pbhs.solar_mass_grams:.2e} \M_sun ({temperature_critical_mass:.2e} grams)")


# PBHs as Dark Matter candidates

| Mass Range (Approximate)  |  Source of Constraint/Window |  Key Observations/Phenomena |
|---|---|---|
| $< 10^{15}$ g ($\sim 10^{-18} M_{\odot}$) | Excluded| Hawking Radiation: Black holes lighter than this mass would have completely evaporated by the present age of the Universe ($\sim 13.8$ billion years).|
| $10^{15} \text{ g} - 10^{20} \text{ g}$ | Severely Constrained | Gamma-ray/CMB distortions, Big Bang Nucleosynthesis (BBN): Evaporation of PBHs in this range before or during BBN or reionization would inject high-energy particles, altering the observed light element abundances and the Cosmic Microwave Background (CMB) spectrum. |
| $\sim 10^{20} \text{ g} - 10^{24} \text{ g}$ ($\sim 10^{-13} - 10^{-9} M_{\odot}$, Asteroid Mass) | Open Window | This range is currently one of the most unconstrained, often referred to as the asteroid-mass window. Constraints from microlensing and other effects are weaker here. |
| $\sim 10^{-9} M_{\odot} - 10^{-2} M_{\odot}$ | Severely Constrained | Microlensing Surveys (e.g., MACHO, EROS): Observing the temporary brightening of distant stars due to the gravitational lensing effect of massive objects (PBHs) passing in front of them provides strong limits on PBHs in this range (up to a few $M_{\odot}$). |
| $\sim 10 M_{\odot} - 100 M_{\odot}$ (Stellar Mass)| Controversial/Reopened Window | Gravitational Waves (LIGO/Virgo/KAGRA): PBH mergers in this range are consistent with the observed merger rates, re-opening this window, but constraints from microlensing and dynamical friction are still debated and may require a non-monochromatic (extended) mass distribution. |
| $\sim 10^3 M_{\odot} - 10^5 M_{\odot}$ | Constrained | Dynamical constraints, effects on stellar clusters: Massive PBHs would affect the dynamics of globular clusters and ultra-faint dwarf galaxies. |
|$\gg 10^5 M_{\odot}$ | Severely Constrained | CMB, Large-Scale Structure (LSS): Very massive PBHs would significantly perturb the LSS formation and induce too many anisotropies in the CMB. |



# The Expected PBH Mass Distribution

If PBHs account for all DM, their distribution in mass, $\psi(M)$, is not expected to be monochromatic (all PBHs having the same mass) due to the nature of their formation from primordial density fluctuations.

The most commonly considered theoretical mass distribution for PBHs formed from a smooth, narrow peak in the primordial power spectrum is the log-normal distribution:

$$\psi(M) \propto \frac{1}{M \sigma} \exp \left( -\frac{\ln^2(M/M_c)}{2\sigma^2} \right)$$

* $M$: PBH mass
* $M_c$: Peak (characteristic) mass of the distribution
* $\sigma$: Width of the distribution

This extended distribution means that if the peak mass $M_c$ falls in an "open window" (like the asteroid-mass range), the tails of the distribution must still satisfy the constraints in other mass ranges.

The recent consensus, especially motivated by Gravitational Wave observations and the need to evade various constraints, suggests that a PBH DM scenario requires a broad mass distribution (large $\sigma$) that spans several orders of magnitude, with the majority of the mass concentrated in the open windows, such as the asteroid-mass window ($\sim 10^{20} - 10^{24} \text{ g}$), and possibly the stellar-mass window ($\sim 10 - 100 M_{\odot}$).

In summary, the "calculation" points towards an allowed mass distribution that is:

* Entirely above $\sim 10^{15}$ g to avoid complete evaporation.
* Highly suppressed in the ranges strongly constrained by microlensing ($10^{-2} M_{\odot} - 10 M_{\odot}$).
* Dominated by mass in the open windows, particularly the asteroid-mass range, to constitute the majority of dark matter.

# Asteroid-mass window as source of all dark matter

If Primordial Black Holes (PBHs) in the asteroid-mass window are the source of all dark matter, the required number density in a volume of 1 cubic light-year (ly³) is surprisingly small.

The calculation involves two main steps:
1. Determine the local dark matter mass density ($\rho_{\text{DM}}$)
2. Divide the mass density by the PBH mass (1$M_{\text{PBH}}$) to find the number density (2$n_{\text{PBH}}$)

## Local Dark Matter Mass Density ($\rho_{\text{DM}}$)

The mass density of dark matter in the solar neighborhood is widely estimated to be:

$$\rho_{\text{DM}} \approx 0.3 - 0.4 \text{ GeV/cm}^3$$

To translate this into solar masses per cubic light-year, we use the following conversions:

* $1 \text{ GeV/cm}^3 \approx 5.35 \times 10^{-4} M_{\odot}/\text{pc}^3$
* $1 \text{ pc} \approx 3.26 \text{ ly}$
* $1 \text{ pc}^3 \approx 3.26^3 \text{ ly}^3 \approx 34.6 \text{ ly}^3$

Using a fiducial value of $\rho_{\text{DM}} \approx 0.4 \text{ GeV/cm}^3$:

$$\rho_{\text{DM}} \approx \frac{0.4 \text{ GeV/cm}^3 \times 5.35 \times 10^{-4} M_{\odot}/\text{pc}^3}{1 \text{ GeV/cm}^3} \approx 2.14 \times 10^{-4} M_{\odot}/\text{pc}^3$$

Now, convert to $M_{\odot}/\text{ly}^3$:

$$\rho_{\text{DM}} \approx \frac{2.14 \times 10^{-4} M_{\odot}}{\text{pc}^3} \times \frac{1 \text{ pc}^3}{34.6 \text{ ly}^3}$$

$$\mathbf{\rho_{\text{DM}} \approx 6.2 \times 10^{-6} \text{ Solar Masses per Cubic Light-Year}}$$

## PBH Number Density ($n_{\text{PBH}}$)

The asteroid-mass window is typically cited as $10^{17} \text{ g}$ to $10^{23} \text{ g}$ .

We'll calculate the number of PBHs per cubic light-year ($N_{\text{PBH}}$) for the extremes of this range and a central value.

The formula for the number density is:

$$N_{\text{PBH}} = \frac{\rho_{\text{DM}} \text{ (in kg/ly}^3)}{M_{\text{PBH}} \text{ (in kg)}}$$

First, convert the dark matter mass to kilograms:

* $1 M_{\odot} \approx 2 \times 10^{30} \text{ kg}$

* $\rho_{\text{DM}} \approx 6.2 \times 10^{-6} M_{\odot}/\text{ly}^3 \approx (6.2 \times 10^{-6}) \times (2 \times 10^{30}) \text{ kg}/\text{ly}^3 \approx \mathbf{1.24 \times 10^{25} \text{ kg/ly}^3}$

### Resulting Distribution

| PBH Mass (MPBH​) | PBH Mass in Kilograms | Number of PBHs per ly³ (NPBH​) |
|---|---|---|
|Lower End ($10^{17} \text{ g}$)     | $10^{14} \text{ kg}$ | $\frac{1.24 \times 10^{25} \text{ kg/ly}^3}{10^{14} \text{ kg}} \approx \mathbf{1.24 \times 10^{11}}$ (124 Billion)
|Central Value ($10^{20} \text{ g}$) | $10^{17} \text{ kg}$ | $\frac{1.24 \times 10^{25} \text{ kg/ly}^3}{10^{17} \text{ kg}} \approx \mathbf{1.24 \times 10^8}$ (124 Million)
|Upper End ($10^{23} \text{ g}$)     | $10^{20} \text{ kg}$ | $\frac{1.24 \times 10^{25} \text{ kg/ly}^3}{10^{20} \text{ kg}} \approx \mathbf{124,000}$


### Conclusion

For PBHs in the asteroid-mass window to account for all of the dark matter, their number density in a single cubic light-year of space must be between approximately 124,000 (for the heaviest PBHs) and 124 billion (for the lightest PBHs).This high number density suggests that the PBHs would be extremely numerous but individually very light compared to typical astrophysical objects. They would be spread out but constantly zipping through the Solar System and surrounding space, which is why future missions are looking for subtle gravitational perturbations from these hypothetical objects.



# fitar os pbhs no halo da galaxia
# analogia com 21cm, emissao hawking
# Onda gravitacional
# Efeitos estocasticos?
# Distancia media
# ja foi feito por alguem?

# Accepted Milky Way Dark Matter Halo Parameters

The distribution and size of the Milky Way's dark matter (DM) halo are not uniquely defined, as different models (cored vs. cuspy) and observational tracers yield variations. However, a Navarro-Frenk-White (NFW) profile is the most widely cited in the $\Lambda$ CDM framework, while the Burkert profile is often favored by fits to observed rotation curves in low-surface-brightness and some high-surface-brightness galaxies, including the Milky Way.

## Canonical Halo Model (for calculation)

For our insights, we will adopt the standard NFW parameters and the most common local DM density estimate, as these are frequently used in PBH constraint literature.

* Halo Profile (NFW): The mass density $\rho_{\text{DM}}(r)$ at a galactocentric radius $r$ is given by:

$$\rho_{\text{NFW}}(r) = \frac{\rho_s}{\frac{r}{r_s} \left(1 + \frac{r}{r_s}\right)^2}$$

where $\rho_s$ is the characteristic density and $r_s$ is the scale radius.

* Local Dark Matter Density ($\rho_\odot$): The accepted value at the Sun's position ($R_\odot \approx 8.122$ kpc) is approximately:

$$\rho_{\text{DM}}(R_\odot) \approx 0.3 - 0.5 \text{ GeV/cm}^3 \approx 4.0 - 6.7 \times 10^{-22} \text{ g/cm}^3$$

We will use a canonical value of:

$$\mathbf{\rho_\odot = 0.4 \text{ GeV/cm}^3} \approx 6.75 \times 10^{-25} \text{ g/cm}^3$$

Note: The NFW scale radius ($r_s$) for the Milky Way is typically $\sim 15 - 20$ kpc, and the virial radius ($R_{\text{vir}}$, where the mean density is $200\times\rho_{\text{crit}}$) is $\sim 200$ kpc, enclosing a total mass $M_{\text{vir}} \sim 10^{12} M_\odot$.

TODO: Some math/graphics here to better understand it.

 # Primordial Black Hole Number Density
 
 Assume that $\mathbf{100\%}$ of the dark matter mass density is composed of monochromatic PBHs of mass $\mathbf{M_{\text{PBH}}}$. The asteroid-mass window for PBHs to constitute all of the DM is typically taken as $10^{17} \text{ g} \lesssim M_{\text{PBH}} \lesssim 10^{23} \text{ g}$ (based primarily on constraints from microlensing and constraints related to the disruption of objects in the early universe), but a canonical value for calculation is often $\mathbf{M_{\text{PBH}} = 10^{20} \text{ g}}$.


 ## A) Mass Conversion
 
 First, we convert the canonical local mass density into the physical units we're using:$$\rho_{\text{DM}} = 0.4 \frac{\text{GeV}}{\text{cm}^3} \times \frac{1}{0.938 \times 10^9} \frac{\text{g}}{\text{GeV/c}^2} \times \frac{c^2}{1} \approx 6.75 \times 10^{-25} \frac{\text{g}}{\text{cm}^3}$$
 
 (Self-Correction: It is standard in DM literature to use $\text{GeV}/\text{cm}^3$ as a mass density, implicitly assuming $c=1$, or, more strictly, referring to the rest mass density. $1 \text{ GeV}/c^2 \approx 1.78 \times 10^{-24} \text{ g}$.)
 
 Let's use the strict conversion for precision:$$\rho_{\text{DM}} = 0.4 \frac{\text{GeV}}{c^2 \cdot \text{cm}^3} \times \frac{1.7827 \times 10^{-24} \text{ g}}{1 \text{ GeV}/c^2} \approx \mathbf{7.13 \times 10^{-25} \text{ g/cm}^3}$$

## B) Number Density Calculation

The number density $\mathbf{n_{\text{PBH}}(r)}$ of PBHs at a given radius $r$ is simply the mass density divided by the mass of a single PBH:

$$n_{\text{PBH}}(r) = \frac{\rho_{\text{DM}}(r)}{M_{\text{PBH}}}$$

At the Solar Neighborhood ($R=R_\odot$):

$$\mathbf{n_{\text{PBH}}}(\mathbf{R_\odot}) = \frac{7.13 \times 10^{-25} \text{ g/cm}^3}{10^{20} \text{ g}} = \mathbf{7.13 \times 10^{-45} \text{ cm}^{-3}}$$

To use more convenient units (e.g., cubic Astronomical Units), we convert:

$$1 \text{ AU} \approx 1.496 \times 10^{13} \text{ cm}$$

$$1 \text{ AU}^3 \approx (1.496 \times 10^{13})^3 \text{ cm}^3 \approx 3.348 \times 10^{39} \text{ cm}^3$$

$$\mathbf{n_{\text{PBH}}}(\mathbf{R_\odot}) \approx (7.13 \times 10^{-45} \text{ cm}^{-3}) \times (3.348 \times 10^{39} \text{ cm}^3/\text{AU}^3)$$

$$\mathbf{n_{\text{PBH}}}(\mathbf{R_\odot}) \approx \mathbf{2.39 \times 10^{-5} \text{ PBH/AU}^3}$$



# Characteristic PBH Separation

The characteristic average separation distance $\mathbf{d}$ between PBHs can be estimated by considering the volume occupied by one PBH, assuming a uniform, non-clustered distribution:

$$d \approx \left(\frac{1}{n_{\text{PBH}}}\right)^{1/3}$$

At the Solar Neighborhood:

$$d_{\odot} \approx \left(\frac{1}{7.13 \times 10^{-45} \text{ cm}^{-3}}\right)^{1/3} \approx (1.40 \times 10^{44} \text{ cm}^3)^{1/3}$$$$d_{\odot} \approx \mathbf{5.19 \times 10^{14} \text{ cm}}$$

Converting to more relatable distance units:

$$1 \text{ parsec (pc)} \approx 3.086 \times 10^{18} \text{ cm}$$

$$1 \text{ AU} \approx 1.496 \times 10^{13} \text{ cm}$$$$\mathbf{d_{\odot}} \approx \frac{5.19 \times 10^{14} \text{ cm}}{1.496 \times 10^{13} \text{ cm/AU}} \approx \mathbf{34.7 \text{ AU}}$$

$$\mathbf{d_{\odot}} \approx \frac{5.19 \times 10^{14} \text{ cm}}{3.086 \times 10^{18} \text{ cm/pc}} \approx \mathbf{1.68 \times 10^{-4} \text{ pc}}$$

Insights:

* Local PBH Spacing: The asteroid-mass PBHs ($10^{20} \text{ g}$) are, on average, separated by approximately $35 \text{ AU}$ in the solar neighborhood. This is roughly the distance to Pluto (mean distance $\sim 40 \text{ AU}$).
* Implications for Direct Encounter: The nearest PBH is likely within $100 \text{ AU}$. This proximity, combined with typical halo velocities, leads to a high frequency of gravitational "scattering" or weak close encounters with Solar System objects over the system's lifetime, which is a key physical constraint on the PBH parameter space.
* Halo Density Profile: The key insight from the density profile $\rho_{\text{DM}}(r)$ is that the number density $\mathbf{n_{\text{PBH}}(r)}$ and characteristic separation $\mathbf{d(r)}$ will vary radially:
$$\rho_{\text{NFW}}(r) \propto 1/r \quad \text{for } r \ll r_s \text{ (cusp)}$$
$$\rho_{\text{NFW}}(r) \propto 1/r^3 \quad \text{for } r \gg r_s$$

This means the PBHs are much more densely packed in the galactic center (where $d$ is smaller) and much more loosely spread in the far outer halo (where $d$ is larger). If we use the NFW scaling for $r \ll 1 \text{ kpc}$ (assuming the cusp holds), the number density could be orders of magnitude higher, and the spacing correspondingly smaller. The concentration parameter and scale radius govern this exact scaling.


# Does this distribution answer the galactic rotation curve problem?

No, the distribution of primordial black holes (PBHs) in the dark matter halo, as defined by a canonical density profile (like NFW or Burkert), does not solve the galactic rotation curve problem; it is the solution, or rather, the representation of the solution, in the context of the Cold Dark Matter ($\Lambda$CDM) paradigm.The rotation curve problem is the empirical observation that the rotational velocity $V_c(r)$ of matter in galaxies remains flat, or even slightly increases, at large galactocentric radii $r$, defying the expected Keplerian fall-off $V_c \propto r^{-1/2}$ based on the visible matter distribution alone. This implies that the total enclosed mass 1$M(r)$ grows linearly with radius (2$M(r) \propto r$) in the outer parts, which requires a component of non-luminous, extended matter—the dark matter halo.



## Key Insights

1) Mass Distribution is the Solution: The theoretical density profile $\rho_{\text{DM}}(r)$ (e.g., NFW, Burkert) that generates the flat rotation curve is the solution to the rotation curve problem. Whether the mass density $\rho_{\text{DM}}(r)$ is comprised of PBHs, WIMPs, or axions is a question of composition, not distribution. As long as the PBH mass $M_{\text{PBH}}$ is small enough (like your $10^{20} \text{ g}$ assumption) that they behave as cold, collisionless particles—which they do—their collective gravitational effect is indistinguishable from any other dark matter candidate with the same mass density profile.

2) Asteroid-Mass PBHs and Collisionless Dynamics: For PBHs in the asteroid-mass range, the particle number density is extremely high ($n_{\text{PBH}} \approx 2.4 \times 10^{-5} \text{ AU}^{-3}$ locally), meaning the physical separation (tens of AU) is vast compared to their Schwarzschild radii ($R_S \approx 10^{-10} \text{ cm}$). They satisfy the criteria for a collisionless fluid—their gravitational self-interaction cross-section is negligible compared to the gravitational effects of the entire galaxy. Therefore, they fall into the gravitational potential of the galaxy exactly as other CDM candidates do, tracing the required mass distribution $\rho_{\text{DM}}(r)$ that flattens the rotation curve.

The Only Constraint is the Mass Profile: The only thing required to match the observed galactic rotation curve $V_c(r)$ is that the total mass density $\rho_{\text{tot}}(r) = \rho_{\text{baryons}}(r) + \rho_{\text{DM}}(r)$ satisfies the centrifugal equilibrium equation:$$\frac{V_c^2(r)}{r} = \frac{G M(r)}{r^2}$$

where $M(r) = 4\pi \int_0^r \rho_{\text{tot}}(r') r'^2 dr'$. Any candidate that constitutes the $\rho_{\text{DM}}(r)$ required by this equation solves the problem by definition.

In summary, the asteroid-mass PBH hypothesis uses the established dark matter distribution (the solution to the rotation curve problem) and proposes a specific object for its composition.




# how can we extend your solution for PBHs distribution for PBHs actually solve the galaxy rotation problem?

The question of how the primordial black hole (PBH) distribution can solve the galactic rotation curve problem is a subtle one, as the standard cosmological model ($\Lambda$CDM) already "solves" it by postulating a collisionless dark matter halo with a specific density profile $\rho_{\text{DM}}(r)$. Since asteroid-mass PBHs act as a cold, collisionless dark fluid, their presence does constitute the dark matter mass density $\rho_{\text{DM}}(r)$, thus explaining the flat rotation curve.However, if you are looking for ways the PBH distribution itself could provide an extension or refinement to the standard solution, we must look at the small-scale issues of $\Lambda$CDM.


#  Refinement on Small Scales: The Core-Cusp Problem

The most significant refinement or extension that PBHs could provide relates to the Core-Cusp Problem. This is a small-scale structure discrepancy, a key point of tension in the standard $\Lambda$CDM model:

## The Cusp-Core Discrepancy

* Prediction (N-body Simulations): 1$\Lambda$CDM N-body simulations typically predict that dark matter halos should have a cuspy density profile, such as the Navarro-Frenk-White (NFW) profile, where the density diverges as 2$\rho_{\text{NFW}}(r) \propto r^{-1}$ for 3$r \to 0$.4
* Observation (Dwarf Galaxies): Rotation curve observations, particularly in low-surface-brightness (LSB) and dwarf galaxies, strongly favor a cored profile (e.g., Burkert profile), where the density flattens to a constant $\rho_{\text{Burkert}}(r) \to \rho_0$ for $r \to 0$.

## The PBH Extension

The PBH hypothesis offers a non-baryonic mechanism to alleviate this problem, beyond the usual recourse to baryonic feedback (supernovae, AGN outflows).
1) Dynamical Friction and Scattering: If the PBH masses are large enough ($\gtrsim 10^3 M_{\odot}$), the discrete, compact nature of the dark matter particles becomes important. $\text{PBH}-\text{PBH}$ two-body interactions, or dynamical friction of PBHs on the central galactic structure, could potentially drive energy away from the center, effectively:
* Smoothing out the Cusp: Gravitational scattering of massive PBHs in the dense inner regions could transfer kinetic energy to the system, causing the PBHs to "puff up" the center and transition the density profile from a steep cusp to a shallower core.
2) Early Structure Formation (Poisson Fluctuations): The Poisson fluctuations in the initial number density of PBHs are much larger than the Gaussian fluctuations of a standard WIMP fluid. These large initial fluctuations can lead to the earlier formation of smaller, dense PBH mini-halos at high redshift. The subsequent merger and relaxation of these structures could influence the final density profile of the larger galactic halo, potentially leading to a profile that is closer to a core than a cusp.

# The Extreme Extension: PBHs and Modified Gravity (MOND)

A more radical "extension" would be to argue that the PBH distribution only appears to solve the problem when the mass-to-light ratios of baryonic matter are underestimated, and that the true solution lies in a modification of gravity that is seeded or triggered by the PBH mass scale.
While there is no established theory where the mere presence of PBHs causes a modified Newtonian dynamics (MOND) acceleration, one could postulate:

* PBH-Induced Crossover Scale: If PBHs dominate the vacuum energy or couple to a light scalar field that mediates a fifth force (a la Scalar-Tensor-Vector Gravity or STVG/MOG), the transition scale from Newtonian to modified dynamics ($a_0 \approx 1.2 \times 10^{-10} \text{ m/s}^2$) could be physically linked to the PBH mass or clustering properties. This approach removes the need for a non-baryonic halo and attributes the flat rotation curve to an alteration of the gravitational field equation itself at low accelerations.

In summary, for PBHs to provide a non-trivial solution beyond the standard CDM picture, their distribution and kinematics must deviate from a smooth fluid, which is only physically plausible if:

1)They solve the Core-Cusp Problem (a deviation from the NFW cusp) through discrete-particle dynamics like $\text{PBH}-\text{PBH}$ scattering or early structure formation.

2) They are the physical source or mediator for a Modified Gravity theory (an extremely speculative but complete "solution" to the rotation curve).